In [1]:
import numpy as np
import os
from os.path import expanduser
home_dir = expanduser("~")
module_path = home_dir + '/modules/'
import sys
sys.path.append(module_path)
import pandas as pd
import math
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from numba import jit
import time, datetime
%load_ext autoreload
%autoreload 1
%aimport environmental_density
from environmental_density import get_density_periodic
np.random.seed(999)

In [2]:
galfile = pd.read_hdf('/scratch/data/galcats/P200/galaxies.Z01.h5')
#galfile = pd.read_hdf('test_galcat.h5')
galaxies = galfile.as_matrix()
gal_header = galfile.keys().tolist()
print(gal_header)

### Remove data points with halo mass below 10.5
print(np.shape(galaxies))
galaxies = galaxies[galaxies[:,6] > 10.5, :]
print(np.shape(galaxies))

['X_pos', 'Y_pos', 'Z_pos', 'X_vel', 'Y_vel', 'Z_vel', 'Halo_mass', 'Stellar_mass', 'SFR', 'Intra_cluster_mass', 'Halo_mass_peak', 'Stellar_mass_obs', 'SFR_obs', 'Halo_radius', 'Concentration', 'Halo_spin', 'Scale_peak_mass', 'Scale_half_mass', 'Scale_last_MajM', 'Type']
(594946, 20)
(306925, 20)


In [3]:
data_dict = {'X_pos': 0, 'Y_pos': 1, 'Z_pos': 2, 'X_vel': 3, 'Y_vel': 4, 'Z_vel': 5, 'Halo_mass': 6, 
             'Stellar_mass': 7, 'SFR': 8, 'Intra_cluster_mass': 9, 'Halo_mass_peak': 10, 'Stellar_mass_obs': 11, 
             'SFR_obs': 12, 'Halo_radius': 13, 'Concentration': 14, 'Halo_spin': 15, 'Scale_peak_mass': 16, 
             'Scale_half_mass': 17, 'Scale_last_MajM': 18, 'Type': 19}

In [4]:
### Make sure that the edges are what they should be
print(np.min(galaxies[:, 0]), np.max(galaxies[:, 0]))
print(np.min(galaxies[:, 1]), np.max(galaxies[:, 1]))
print(np.min(galaxies[:, 2]), np.max(galaxies[:, 2]))

0.0008115685 199.99994
0.0015050907 199.99956
0.00092961494 199.99817


In [5]:
coordinates = galaxies[:, :3]
halo_masses = np.power(10, galaxies[:, 6])
nr_points = np.shape(coordinates)[0]

In [7]:
nr_neighbours_wanted = 11
box_sides = np.array([200, 200, 200])

neigh_densities = get_density_periodic(coordinates, halo_masses, nr_neighbours_wanted, 
                                        box_sides, nr_points, verbatim=True, progress_file=True)
neigh_densities = np.log10(neigh_densities)

14:20:42      Time elapsed: 0.30min. Time remaining: 92.41min.
14:21:00      Time elapsed: 0.60min. Time remaining: 90.92min.
14:21:17      Time elapsed: 0.89min. Time remaining: 90.30min.
14:21:36      Time elapsed: 1.19min. Time remaining: 90.37min.
14:21:54      Time elapsed: 1.50min. Time remaining: 90.55min.
14:22:12      Time elapsed: 1.81min. Time remaining: 90.58min.
14:22:31      Time elapsed: 2.12min. Time remaining: 90.77min.
14:22:50      Time elapsed: 2.43min. Time remaining: 90.66min.
14:23:08      Time elapsed: 2.73min. Time remaining: 90.44min.
14:23:28      Time elapsed: 3.06min. Time remaining: 90.89min.
14:23:46      Time elapsed: 3.37min. Time remaining: 90.64min.
14:24:04      Time elapsed: 3.67min. Time remaining: 90.26min.
14:24:23      Time elapsed: 3.98min. Time remaining: 90.01min.
14:24:41      Time elapsed: 4.28min. Time remaining: 89.64min.
14:25:00      Time elapsed: 4.59min. Time remaining: 89.37min.
14:25:18      Time elapsed: 4.91min. Time remaining: 89

15:02:17      Time elapsed: 41.88min. Time remaining: 57.00min.
15:02:39      Time elapsed: 42.25min. Time remaining: 56.74min.
15:03:01      Time elapsed: 42.61min. Time remaining: 56.47min.
15:03:23      Time elapsed: 42.98min. Time remaining: 56.20min.
15:03:45      Time elapsed: 43.34min. Time remaining: 55.93min.
15:04:07      Time elapsed: 43.72min. Time remaining: 55.68min.
15:04:30      Time elapsed: 44.09min. Time remaining: 55.42min.
15:04:52      Time elapsed: 44.47min. Time remaining: 55.16min.
15:05:15      Time elapsed: 44.84min. Time remaining: 54.89min.
15:05:37      Time elapsed: 45.22min. Time remaining: 54.63min.
15:06:00      Time elapsed: 45.59min. Time remaining: 54.36min.
15:06:22      Time elapsed: 45.97min. Time remaining: 54.09min.
15:06:45      Time elapsed: 46.34min. Time remaining: 53.82min.
15:07:07      Time elapsed: 46.72min. Time remaining: 53.55min.
15:07:30      Time elapsed: 47.09min. Time remaining: 53.28min.
15:07:52      Time elapsed: 47.47min. Ti

15:50:24      Time elapsed: 90.00min. Time remaining: 16.65min.
15:50:47      Time elapsed: 90.38min. Time remaining: 16.31min.
15:51:09      Time elapsed: 90.75min. Time remaining: 15.97min.
15:51:32      Time elapsed: 91.13min. Time remaining: 15.63min.
15:51:54      Time elapsed: 91.51min. Time remaining: 15.28min.
15:52:17      Time elapsed: 91.88min. Time remaining: 14.94min.
15:52:40      Time elapsed: 92.26min. Time remaining: 14.60min.
15:53:02      Time elapsed: 92.64min. Time remaining: 14.25min.
15:53:25      Time elapsed: 93.02min. Time remaining: 13.91min.
15:53:48      Time elapsed: 93.39min. Time remaining: 13.56min.
15:54:10      Time elapsed: 93.77min. Time remaining: 13.22min.
15:54:33      Time elapsed: 94.15min. Time remaining: 12.88min.
15:54:55      Time elapsed: 94.52min. Time remaining: 12.53min.
15:55:18      Time elapsed: 94.90min. Time remaining: 12.19min.
15:55:41      Time elapsed: 95.28min. Time remaining: 11.84min.
15:56:03      Time elapsed: 95.65min. Ti

In [ ]:
print(np.mean(neigh_densities))
print(np.sum(halo_masses)/200**3)

# Make a new pd dataset and save it

In [8]:
pd_dataframe = pd.DataFrame(data=galaxies, columns=gal_header)
pd_dataframe['Environmental_density'] = neigh_densities

In [9]:
pd_dataframe.to_hdf('test_galcat_w_log_densities_3e5.h5', 'w')

# Try loading the newly created galaxy catalogue

In [10]:
galfile = pd.read_hdf('test_galcat_w_log_densities_3e5.h5')
galaxies = galfile.as_matrix()
gal_header = galfile.keys().tolist()
print(gal_header)
print(np.shape(galaxies))

['X_pos', 'Y_pos', 'Z_pos', 'X_vel', 'Y_vel', 'Z_vel', 'Halo_mass', 'Stellar_mass', 'SFR', 'Intra_cluster_mass', 'Halo_mass_peak', 'Stellar_mass_obs', 'SFR_obs', 'Halo_radius', 'Concentration', 'Halo_spin', 'Scale_peak_mass', 'Scale_half_mass', 'Scale_last_MajM', 'Type', 'Environmental_density']
(306925, 21)


In [11]:
print('%.2f' % (np.mean(galaxies[:,20])))
print('%.2f' % (np.amax(galaxies[:,20])))
print('%.2f' % (np.amin(galaxies[:,20])))
#print(np.sum(galaxies[:,6]) / 200**3)

10.90
17.92
8.15


In [ ]:
galaxies[:,20] = np.log10(galaxies[:,20])

# Old functions

In [ ]:
@jit(nopython=True)
def nearest_k_neighbours_1(coordinates, nr_neighbours, box_sides, nr_points):
    indices_of_neighbors = np.zeros((coordinates.shape[0], nr_neighbours))
    for i in range(nr_points):

        euclidean_dists = np.sqrt(np.sum(np.minimum(np.abs(coordinates - coordinates[i,:]), box_sides - 
                                                    (np.abs(coordinates - coordinates[i,:])))**2, 1))
        indices_of_neighbors[i,:] = np.argsort(euclidean_dists)[:nr_neighbours]
    
    return indices_of_neighbors

In [ ]:
@jit(nopython=True)
def nearest_k_neighbours_2(coordinates, nr_neighbours, box_sides, nr_points):
    indices_of_neighbors = np.zeros((coordinates.shape[0], nr_neighbours))
#    weights_of_neighbors = np.zeros((coordinates.shape[0], nr_neighbours))
    for i in range(nr_points):
        norm_dist = np.abs(coordinates - coordinates[i,:])
        inv_dist = box_sides - norm_dist
        shortest_dists = np.minimum(norm_dist, inv_dist)
        squared_shortest_dists = shortest_dists**2
        summed_squared_shortest_dists = np.sum(squared_shortest_dists, 1)
        euclidean_dists = np.sqrt(summed_squared_shortest_dists)
        
        indices_of_neighbors[i,:] = np.argsort(euclidean_dists)[:nr_neighbours]
#        weights_of_neighbors[i,:]
    
    return indices_of_neighbors

In [ ]:
def nearest_k_neighbours_3(coordinates, nr_neighbours, box_sides, nr_points):
    indices_of_neighbors = np.zeros((coordinates.shape[0], nr_neighbours))
    for i in range(nr_points):

        euclidean_dists = np.sqrt(np.sum(np.minimum(np.abs(coordinates - coordinates[i,:]), box_sides - 
                                                    (np.abs(coordinates - coordinates[i,:])))**2, 1))
        indices_of_neighbors[i,:] = np.argpartition(euclidean_dists, nr_neighbours)[:nr_neighbours]
    
    return indices_of_neighbors

In [ ]:
@jit
def get_density_periodic_jit(coordinates, weights, nr_neighbours, box_sides, nr_points):
    indices_of_neighbors = np.zeros((coordinates.shape[0], nr_neighbours), dtype=np.int8)
    masses_of_neighbors = np.zeros((coordinates.shape[0], nr_neighbours))
    neigh_densities = np.zeros(nr_points)
    for i in range(nr_points):
        norm_dist = np.abs(coordinates - coordinates[i,:])
        inv_dist = box_sides - norm_dist
        shortest_dists = np.minimum(norm_dist, inv_dist)
        squared_shortest_dists = shortest_dists**2
        summed_squared_shortest_dists = np.sum(squared_shortest_dists, 1)
        euclidean_dists = np.sqrt(summed_squared_shortest_dists)
        
        indices_of_k_closest_neighbours = np.argpartition(euclidean_dists, nr_neighbours)[:nr_neighbours]
        indices_of_neighbors[i,:] = indices_of_k_closest_neighbours
        masses_of_neighbors[i,:] = weights[indices_of_k_closest_neighbours]
        
        sphere_radius = np.max(euclidean_dists[indices_of_k_closest_neighbours])
        print('sphere radius: ', sphere_radius)
        sphere_volume = (4/3) * math.pi * sphere_radius**3
#        print('sphere volume: ', sphere_volume)
        density =  np.sum(masses_of_neighbors[i,:]) / sphere_volume
#        print('density: ', density)
        neigh_densities[i] = density
    
    return indices_of_neighbors, masses_of_neighbors, neigh_densities
                

In [ ]:
#@jit
def get_density_periodic_OLD(coordinates, weights, nr_neighbours, box_sides, nr_points, verbatim=False):
    
    with open('results.txt', 'w+') as f:

        neigh_densities = np.zeros(nr_points)

        start = time.time()
        for i in range(nr_points):

            if int(i/1000) == i/1000 and i > 0:
                end = time.time()
                elapsed_time = (end-start)/60
                time_remaining = elapsed_time / i * (nr_points - i)
                if verbatim:
                    print('%s      Time elapsed: %.2fmin. Time remaining: %.2fmin.' % (datetime.datetime.now().strftime("%H:%M:%S"),
                                                                            elapsed_time, time_remaining))
                f.write('%s      Time elapsed: %.2fmin. Time remaining: %.2fmin.\n' % (datetime.datetime.now().strftime("%H:%M:%S"),
                                                                        elapsed_time, time_remaining))
                f.flush()
            norm_dist = np.abs(coordinates - coordinates[i,:])
            inv_dist = box_sides - norm_dist
            shortest_dists = np.minimum(norm_dist, inv_dist)
            squared_shortest_dists = shortest_dists**2
            summed_squared_shortest_dists = np.sum(squared_shortest_dists, 1)
            euclidean_dists = np.sqrt(summed_squared_shortest_dists)

            indices_of_k_closest_neighbours = np.argpartition(euclidean_dists, nr_neighbours)[:nr_neighbours]
            masses_of_k_closest_neighbours = weights[indices_of_k_closest_neighbours]

            sphere_radius = np.amax(euclidean_dists[indices_of_k_closest_neighbours])

            if sphere_radius > box_sides[0]/2:
                print('WARNING! Sphere with radius larger than %d mega parsecs created.' % (box_sides[0]/2))
                f.write('WARNING! Sphere with radius larger than %d mega parsecs created.\n' % (box_sides[0]/2))
                f.flush()
            sphere_volume = (4/3) * math.pi * sphere_radius**3
            density =  np.sum(masses_of_k_closest_neighbours) / sphere_volume
            neigh_densities[i] = density
            
        elapsed_time = (end-start)/60
        time_remaining = elapsed_time / i * (nr_points - i)
        f.write('%s      Script finished. Elapsed time: %.2fmin.' % (datetime.datetime.now().strftime("%H:%M:%S"), elapsed_time))
    
    return neigh_densities